In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping 
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from tensorflow.keras.applications import resnet
from keras.models import *
from keras.layers import *
import keras
from tensorflow import keras
from keras import optimizers
optimizer=keras.optimizers.RMSprop(learning_rate=0.01)
import warnings
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from sklearn.metrics import classification_report
import time
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [ ]:
def data_prep(parasitized,uninfected):

    parasitized= ["dataset/cell_images/cell_images/Parasitized" + '/' +  parasitize for parasitize in parasitized]
    uninfected= ["dataset/cell_images/cell_images/Uninfected" + '/' +  uninfect for uninfect in uninfected]

    labels = len(parasitized)*['parasitized'] + len(uninfected)*['uninfected']
    data = parasitized + uninfected

    return pd.DataFrame({'Image_Path': data , 'Labels': labels})


df = data_prep(os.listdir('dataset/cell_images/cell_images/Parasitized/'), os.listdir('dataset/cell_images/cell_images/Uninfected/'))
df.head()

,Image_Path,Labels
0,dataset/cell_images/cell_images/Parasitized/C1...,parasitized
1,dataset/cell_images/cell_images/Parasitized/C1...,parasitized
2,dataset/cell_images/cell_images/Parasitized/C1...,parasitized
3,dataset/cell_images/cell_images/Parasitized/C1...,parasitized
4,dataset/cell_images/cell_images/Parasitized/C1...,parasitized


In [ ]:
print(os.listdir("dataset/cell_images/cell_images/"))

['Parasitized', 'Uninfected']


In [ ]:
train, test = train_test_split(df, test_size = .30)

img_datagen = ImageDataGenerator(rotation_range=10,
            width_shift_range=0.2, 
            height_shift_range=0.2,
            zoom_range=0.2, 
            horizontal_flip=True,
            fill_mode='nearest',
            brightness_range=[0.2,1.2], 
            validation_split=0.3)

x_train =  img_datagen.flow_from_dataframe(dataframe = train,  x_col='Image_Path', y_col='Labels',  class_mode='binary',target_size=(64,64), shuffle=False, batch_size=10, seed=10, subset='training')
x_test = img_datagen.flow_from_dataframe(dataframe = test,  x_col='Image_Path', y_col='Labels',class_mode='binary',  target_size=(64,64), shuffle=False, batch_size=10, seed=10, subset='validation')
y_test =  x_test.labels

                               
                               
my_callbacks = [
    #tf.keras.callbacks.EarlyStopping(patience=5),
    #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
]

D:\anaconda\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:279: UserWarning: Found 2 invalid image filename(s) in x_col="Image_Path". These filename(s) will be ignored.
  warnings.warn(


Found 13503 validated image filenames belonging to 2 classes.
Found 2480 validated image filenames belonging to 2 classes.


In [ ]:
x_train_full =  img_datagen.flow_from_dataframe(dataframe = df,  x_col='Image_Path', y_col='Labels',  class_mode='binary',target_size=(64,64), shuffle=False, batch_size=10, seed=10, subset='training')

Found 19291 validated image filenames belonging to 2 classes.


D:\anaconda\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:279: UserWarning: Found 2 invalid image filename(s) in x_col="Image_Path". These filename(s) will be ignored.
  warnings.warn(


# ResNet50

In [ ]:
loaded_resnet = load_model("resnet.pkl")

In [ ]:
y_pred = loaded_resnet.predict(x_test)
for i in range(0,2480,1):
    if y_pred[i] >= 0.5:
        y_pred[i]=int(1)
    else:
        y_pred[i]=int(0)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 879,  327],
       [  24, 1250]], dtype=int64)

In [ ]:
from datetime import datetime
import time
time_array=[]
number=[]
for i in range (0,1,1):
    start = time.time()
    y_train_full_pred = loaded_resnet.predict(x_train_full)
    end = time.time()
    time_taken = end - start
    print('Time: ',i+1,time_taken) 
    time_array.append(float(time_taken))
    number.append(i+1)

Time:  1 164.66752290725708


# InceptionV3

In [ ]:
loaded_inception = load_model("inception.pkl")

In [ ]:
y_pred = loaded_inception.predict(x_test)
for i in range(0,2480,1):
    if y_pred[i] >= 0.5:
        y_pred[i]=int(1)
    else:
        y_pred[i]=int(0)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1080,  126],
       [  42, 1232]], dtype=int64)

In [ ]:
from datetime import datetime
time_array=[]
number=[]
for i in range (0,1,1):
    start = time.time()
    y_train_full_pred = loaded_inception.predict(x_train_full)
    end = time.time()
    time_taken = end - start
    print('Time: ',time_taken) 
    time_array.append(float(time_taken))
    number.append(i+1)

Time:  299.3272397518158


# Stacked CNN

In [ ]:
loaded_stacked = load_model("stacked.pkl")

In [ ]:
y_pred = loaded_stacked.predict(x_test)
for i in range(0,2480,1):
    if y_pred[i] >= 0.5:
        y_pred[i]=int(1)
    else:
        y_pred[i]=int(0)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1068,  138],
       [ 142, 1132]], dtype=int64)

In [ ]:
from datetime import datetime
time_array=[]
number=[]
for i in range (0,3,1):
    start = time.time()
    y_train_full_pred = loaded_stacked.predict(x_train_full)
    end = time.time()
    time_taken = end - start
    print('Time: ',i+1,time_taken) 
    time_array.append(float(time_taken))
    number.append(i+1)

Time:  1 78.16479682922363
Time:  2 76.69586443901062
Time:  3 76.31507635116577


# DS CNN

In [ ]:
loaded_custom = load_model("custom.pkl")

In [ ]:
y_pred = loaded_custom.predict(x_test)
for i in range(0,2480,1):
    if y_pred[i] >= 0.5:
        y_pred[i]=int(1)
    else:
        y_pred[i]=int(0)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1066,  140],
       [  47, 1227]], dtype=int64)

In [ ]:
from datetime import datetime
time_array=[]
number=[]
for i in range (0,1,1):
    start = time.time()
    y_train_full_pred = loaded_custom.predict(x_train_full)
    end = time.time()
    time_taken = end - start
    print('Time: ',i+1,time_taken) 
    time_array.append(float(time_taken))
    number.append(i+1)

Time:  1 44.978179931640625
